# Pacbio consensus UMI
This notebook calls consensus muations on PacBio data grouped by by cell_barcode, gene, and UMI. Then, it exports a processed CSV with the following columns:
* cell_barcode
* gene
* UMI
* mutation
* mutation_CCS

In [ ]:
from IPython.display import display

from dms_variants.constants import CBPALETTE

import numpy as np

import pandas as pd

import plotnine as p9

In [ ]:
consensus_UMI_csv = snakemake.input.consensus_UMI_csv
expt = snakemake.wildcards.expt
consensus_gene_csv = snakemake.output.consensus_gene_csv

Style parameters:

In [ ]:
p9.theme_set(p9.theme_classic())

## Load Data

In [ ]:
mutations = pd.read_csv(consensus_UMI_csv)
mutations['mutation'] = mutations['mutation'].fillna('None')
display(mutations)

## Process Data

Generate list of all cell_barcode-gene:

In [ ]:
cb_gene = (
    mutations
    [['cell_barcode',
      'gene']]
    .drop_duplicates()
)
display(cb_gene)

Count total UMI for cell_barcode-gene:

In [ ]:
total_UMI_df = (
    mutations
    .groupby(['cell_barcode', 'gene'])
    ['UMI']
    .nunique()
    .reset_index()
    .rename(columns={'UMI': 'total_UMI'}))
display(total_UMI_df)

Exclude WT UMI:

In [ ]:
mutations_noWT = mutations.query('mutation != "WT"')

display(mutations_noWT)

Count number of UMI supporting each mutation for cell_barcode-gene:

In [ ]:
mutation_UMI_df = (
    mutations_noWT
    .groupby(['cell_barcode', 'gene', 'mutation'])
    ['UMI']
    .nunique()
    .reset_index()
    .rename(columns={'UMI': 'mutation_UMI'}))
display(mutation_UMI_df)

Bring in total UMI counts and calculate fraction of total UMI with each mutation:

In [ ]:
mutation_frac_df = pd.merge(
    left=mutation_UMI_df,
    right=total_UMI_df,
    on=['cell_barcode', 'gene'],
    how='left',
    validate='many_to_one')
mutation_frac_df['frac_UMI'] = (
    mutation_frac_df['mutation_UMI'] /
    mutation_frac_df['total_UMI']
)
display(mutation_frac_df)

**Call Consensus Mutations**  
Label mutation as consensus if it is found in >50% of UMI. This corresponds to `frac_UMI > 0.5` in our dataframe.

In [ ]:
mutation_frac_df['consensus'] = (
    (mutation_frac_df['frac_UMI'] > 0.5))
display(mutation_frac_df)

Plot outcome of consensus calling

In [ ]:
mutation_frac_histo = (
    p9.ggplot(
        (mutation_frac_df
         [['cell_barcode', 'gene', 'mutation', 'frac_UMI', 'consensus']]
         .drop_duplicates()),
         p9.aes(x='frac_UMI',
                fill='consensus')) +
    p9.geom_histogram(bins=20) +
    p9.ggtitle('Mutation fractions\n'
               '(excludes WT UMI)\n'
               f'{expt}') +
    p9.labs(x='fraction of total UMI for cell_barcode-gene') +
    p9.theme(figure_size=(4, 3),
             plot_title=p9.element_text(size=9),
             axis_title=p9.element_text(size=9),
             legend_title=p9.element_text(size=9),
             legend_title_align='center') +
    p9.scale_fill_manual([CBPALETTE[1], CBPALETTE[0]]))

display(mutation_frac_histo)

## Merge mutatant and WT UMIs
Merge data into single dataframe, `gene_mutations`.  
Steps:  
1. Filter `mutation_frac_df` for only consensus mutations
2. Merge with `cb_gene` dataframe so every `cell_barcode-gene` has at least one row.
3. Fill `mutation` column with "WT" if a `cell_barcode-gene` does not have any consensus mutations. 

In [ ]:
# Merge in consesnsus mutations
gene_mutations = pd.merge(
    left=cb_gene,
    right=mutation_frac_df.query('consensus == True'),
    on=['cell_barcode', 'gene'],
    how='left',
    validate='one_to_many'
)

# Fill muttation with WT if no consensus mutation
gene_mutations['mutation'] = (
    gene_mutations['mutation']
    .fillna('WT')
)

display(gene_mutations)

Check that every `cell_barcode-gene-UMI` is represented in final `UMI_mutations` dataframe.

In [ ]:
assert len(gene_mutations[['cell_barcode', 'gene']].drop_duplicates()) == \
    len(cb_gene), "Missing cell_barcode-gene from df"

## Plot outcomes

Plot distribution of total UMI per cell_barcode-gene:

In [ ]:
total_UMI_histo = (
    p9.ggplot(
        (total_UMI_df),
         p9.aes(x='total_UMI')) +
    p9.geom_histogram(bins=20) +
    p9.ggtitle('n UMI per cell_barcode-gene-UMI\n'
               f'{expt}') +
    p9.labs(x='n UMI',
            y='n cell_barcode-gene') +
    p9.theme(figure_size=(4, 3),
             plot_title=p9.element_text(size=9),
             axis_title=p9.element_text(size=9),
             legend_title=p9.element_text(size=9),
             legend_title_align='center'))

display(total_UMI_histo)

Plot distribution of UMI per mutation:

In [ ]:
mutation_UMI_histo = (
    p9.ggplot(
        (mutation_UMI_df
         [['cell_barcode', 'gene', 'mutation', 'mutation_UMI']]
         .drop_duplicates()),
         p9.aes(x='mutation_UMI')) +
    p9.geom_histogram(bins=20) +
    p9.ggtitle('n UMI per mutation\n'
               '(excludes WT UMI)\n'
               f'{expt}') +
    p9.labs(x='n UMI',
            y='n cell_barcode-gene-mutation') +
    p9.theme(figure_size=(4, 3),
             plot_title=p9.element_text(size=9),
             axis_title=p9.element_text(size=9),
             legend_title=p9.element_text(size=9),
             legend_title_align='center'))

display(mutation_UMI_histo)

Plot frac of UMI for each consensus mutation:

In [ ]:
consensus_mutations_histo = (
    p9.ggplot(
        (gene_mutations
         [['cell_barcode', 'gene', 'mutation', 'frac_UMI']]
         .drop_duplicates()),
         p9.aes(x='frac_UMI')) +
    p9.geom_histogram(bins=20) +
    p9.ggtitle('Consensus mutation fractions\n'
               '(excludes WT genes)\n'
               f'{expt}') +
    p9.labs(x='fraction of total UMI for cell_barcode-gene') +
    p9.theme(figure_size=(4, 3),
             plot_title=p9.element_text(size=9),
             axis_title=p9.element_text(size=9),
             legend_title=p9.element_text(size=9),
             legend_title_align='center') +
    p9.scale_color_manual([CBPALETTE[1], CBPALETTE[0]]))

display(consensus_mutations_histo)

## Organize output  
Integrate mutations into genotype

In [ ]:
genotypes = (
    gene_mutations
    .groupby(['cell_barcode', 'gene'])
    ['mutation']
    .unique()
    .reset_index()
    .rename(columns={'mutation': 'consensus_mutations'})
)
genotypes['consensus_mutations'] = [' '.join(map(str, l)) for l in genotypes['consensus_mutations']]

display(genotypes)

Bring in support information (total UMI per `cell_barcode-gene`):

In [ ]:
output_df = pd.merge(
    left=genotypes,
    right=total_UMI_df,
    on=['cell_barcode', 'gene'],
    how='left',
    validate='one_to_one'
)

display(output_df)

Double check that every cell_barcod-gene is represented in final `output_df`

In [ ]:
assert len(output_df[['cell_barcode', 'gene']].drop_duplicates()) == \
    len(cb_gene), "Missing cell_barcode-gene from df"

Make sure no na values are included in final `output_df`. Everything should either have a consensus mutation or be annotated as `"WT"`

In [ ]:
assert output_df.notnull().any().any(), \
    "Found null value in output_df"

In [ ]:
print(f'Saving gene consensus mutations to {consensus_gene_csv}')
output_df.to_csv(consensus_gene_csv, index=False)